# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/installation.md) before you start.

In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
import matplotlib as pl
from PIL import Image
import string_int_label_map_pb2
from google.protobuf import text_format
import glob
import collections
import cv2

# What model to download.
MODEL_NAME = 'trafficsignal'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'trafficsignal.pbtxt')

NUM_CLASSES = 1

IMAGE_SIZE = (12, 8)

TEST_IMAGE_PATHS = (glob.glob("images/imageresize/*.jpg"))

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
print(detection_graph)

def load_labelmap(path):
  with tf.gfile.GFile(path, 'r') as fid:
    label_map_string = fid.read()
    label_map = string_int_label_map_pb2.StringIntLabelMap()
    try:
      text_format.Merge(label_map_string, label_map)
    except text_format.ParseError:
      label_map.ParseFromString(label_map_string)
  _validate_label_map(label_map)
  return label_map

def _validate_label_map(label_map):
  for item in label_map.item:
    if item.id < 1:
      raise ValueError('Label map ids should be >= 1.')
    
def convert_label_map_to_categories(label_map,
                                    max_num_classes,
                                    use_display_name=True):
  categories = []
  list_of_ids_already_added = []
  if not label_map:
    label_id_offset = 1
    for class_id in range(max_num_classes):
      categories.append({
          'id': class_id + label_id_offset,
          'name': 'category_{}'.format(class_id + label_id_offset)
      })
    return categories
  for item in label_map.item:
    if not 0 < item.id <= max_num_classes:
      logging.info('Ignore item %d since it falls outside of requested '
                   'label range.', item.id)
      continue
    if use_display_name and item.HasField('display_name'):
      name = item.display_name
    else:
      name = item.name
    if item.id not in list_of_ids_already_added:
      list_of_ids_already_added.append(item.id)
      categories.append({'id': item.id, 'name': name})
  return categories

def create_category_index(categories):
  category_index = {}
  for cat in categories:
    category_index[cat['id']] = cat
  return category_index

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

label_map = load_labelmap(PATH_TO_LABELS)
categories = convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = create_category_index(categories)


#****************************************************************************************************************#
imgno=0;
RED_MIN1 = np.array([0, 100, 100],np.uint8)
RED_MAX1 = np.array([10, 255, 255],np.uint8)        

RED_MIN2 = np.array([160, 100, 100],np.uint8)
RED_MAX2 = np.array([179, 255, 255],np.uint8)

YELLOW_MIN = np.array([40.0/360*255, 100, 100],np.uint8)
YELLOW_MAX = np.array([66.0/360*255, 255, 255],np.uint8)

GREEN_MIN = np.array([90.0/360*255, 100, 100],np.uint8)
GREEN_MAX = np.array([140.0/360*255, 255, 255],np.uint8)
# Blue color HSV space limits: [ 84  14 211] [104  34 291]
BLUE_MIN = np.array([84,14,211],np.uint8)
BLUE_MAX = np.array([104,34,291],np.uint8)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in TEST_IMAGE_PATHS:
            image = Image.open(image_path)
            image = image.resize((400,300))
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
              [boxes, scores, classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            boxes = np.squeeze(boxes)
            classes = np.squeeze(classes).astype(np.int32)
            scores = np.squeeze(scores)
            box_to_color_map = collections.defaultdict(str)
            min_score_thresh = 0.45
            
            #print( "Image:", imgno, ":",end='')
            #print("Max score:", max(scores),":", "Max score index:", scores.argmax(axis=0), ":",end='')
            #if(max(scores) > min_score_thresh):
            #    print("Score above thresh: ", scores[scores.argmax(axis=0)],end='')
            #print()
            print(max(scores))
            if scores is None or max(scores) > min_score_thresh:
                lightcolor = "UNKNOWN"
                box = tuple(boxes[scores.argmax(axis=0)].tolist())
                #print(box)
                im_width, im_height = image.size
                ymin, xmin, ymax, xmax = box
                (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
                
                light = image.crop((left, top, right, bottom))
                light = light.resize((40,90))
                #light = cv2.inRange(light, RED_MIN1, RED_MAX1) 
                
                toplight = light.crop((0,0,40,30))
                middlelight = light.crop((0,30,40,60))
                bottomlight = light.crop((0,60,40,90))
                
                toplight = np.array(toplight)[:, :, ::-1].copy()
                middlelight = np.array(middlelight)[:, :, ::-1].copy()
                bottomlight = np.array(bottomlight)[:, :, ::-1].copy()
                
                toplight = cv2.cvtColor(toplight,cv2.COLOR_BGR2HSV)
                middlelight = cv2.cvtColor(middlelight,cv2.COLOR_BGR2HSV)
                bottomlight = cv2.cvtColor(bottomlight,cv2.COLOR_BGR2HSV)
                
                # red has hue 0 - 10 & 160 - 180 add another filter 
                # TODO  use Guassian mask
                frame_threshed1 = cv2.inRange(toplight, RED_MIN1, RED_MAX1) 
                frame_threshed2 = cv2.inRange(toplight, RED_MIN2, RED_MAX2)
                print("Image:",imgno,":  ",end='')
                print("Red count:", cv2.countNonZero(frame_threshed1) + cv2.countNonZero(frame_threshed2), ":", end='')
                if cv2.countNonZero(frame_threshed1) + cv2.countNonZero(frame_threshed2) > 50:
                    lightcolor="RED"
                
                frame_threshed3 = cv2.inRange(middlelight, YELLOW_MIN, YELLOW_MAX)
                print("Yellow count:",cv2.countNonZero(frame_threshed3),":",end='')
                if cv2.countNonZero(frame_threshed3) > 50:
                    lightcolor="YELLOW"
               
                frame_threshed4 = cv2.inRange(bottomlight, GREEN_MIN, GREEN_MAX)
                print("Green count:",cv2.countNonZero(frame_threshed4), ":    ", end='')
                if cv2.countNonZero(frame_threshed4) > 50:
                    lightcolor="GREEN"
                print(lightcolor)
                light.save('images/outputresize/lights/'+str(imgno)+"-"+lightcolor+'.jpg')
            imgno=imgno+1

0.978786
Image: 0 :  Red count: 0 :Yellow count: 0 :Green count: 260 :    GREEN
0.994043
Image: 1 :  Red count: 0 :Yellow count: 398 :Green count: 0 :    YELLOW
0.990506
Image: 2 :  Red count: 0 :Yellow count: 360 :Green count: 0 :    YELLOW
0.96605
Image: 3 :  Red count: 0 :Yellow count: 254 :Green count: 0 :    YELLOW
0.96605
Image: 4 :  Red count: 0 :Yellow count: 254 :Green count: 0 :    YELLOW
0.937157
Image: 5 :  Red count: 0 :Yellow count: 384 :Green count: 0 :    YELLOW
0.966418
Image: 6 :  Red count: 0 :Yellow count: 242 :Green count: 0 :    YELLOW
0.992107
Image: 7 :  Red count: 0 :Yellow count: 256 :Green count: 0 :    YELLOW
0.600199
Image: 8 :  Red count: 0 :Yellow count: 279 :Green count: 0 :    YELLOW
0.962218
Image: 9 :  Red count: 0 :Yellow count: 233 :Green count: 0 :    YELLOW
0.952228
Image: 10 :  Red count: 0 :Yellow count: 213 :Green count: 0 :    YELLOW
0.952228
Image: 11 :  Red count: 0 :Yellow count: 213 :Green count: 0 :    YELLOW
0.895322
Image: 12 :  Red cou

0.997611
Image: 124 :  Red count: 155 :Yellow count: 0 :Green count: 0 :    RED
0.997654
Image: 125 :  Red count: 220 :Yellow count: 0 :Green count: 0 :    RED
0.997803
Image: 126 :  Red count: 119 :Yellow count: 0 :Green count: 0 :    RED
0.996876
Image: 127 :  Red count: 275 :Yellow count: 0 :Green count: 0 :    RED
0.999348
Image: 128 :  Red count: 190 :Yellow count: 0 :Green count: 0 :    RED
0.999557
Image: 129 :  Red count: 190 :Yellow count: 0 :Green count: 0 :    RED
0.999677
Image: 130 :  Red count: 210 :Yellow count: 0 :Green count: 0 :    RED
0.999751
Image: 131 :  Red count: 210 :Yellow count: 0 :Green count: 0 :    RED
0.999862
Image: 132 :  Red count: 210 :Yellow count: 0 :Green count: 0 :    RED
0.999848
Image: 133 :  Red count: 260 :Yellow count: 0 :Green count: 0 :    RED
0.999875
Image: 134 :  Red count: 230 :Yellow count: 0 :Green count: 0 :    RED
0.999803
Image: 135 :  Red count: 214 :Yellow count: 0 :Green count: 0 :    RED
0.998204
Image: 136 :  Red count: 234 :Y

0.990159
Image: 225 :  Red count: 0 :Yellow count: 236 :Green count: 0 :    YELLOW
0.892469
Image: 226 :  Red count: 0 :Yellow count: 250 :Green count: 0 :    YELLOW
0.92155
Image: 227 :  Red count: 0 :Yellow count: 254 :Green count: 0 :    YELLOW
0.794261
Image: 228 :  Red count: 0 :Yellow count: 265 :Green count: 0 :    YELLOW
0.920686
Image: 229 :  Red count: 194 :Yellow count: 0 :Green count: 0 :    RED
0.879718
Image: 230 :  Red count: 177 :Yellow count: 0 :Green count: 0 :    RED
0.949355
Image: 231 :  Red count: 209 :Yellow count: 0 :Green count: 0 :    RED
0.981216
Image: 232 :  Red count: 220 :Yellow count: 0 :Green count: 0 :    RED
0.99418
Image: 233 :  Red count: 204 :Yellow count: 0 :Green count: 0 :    RED
0.989009
Image: 234 :  Red count: 204 :Yellow count: 0 :Green count: 0 :    RED
0.989009
Image: 235 :  Red count: 204 :Yellow count: 0 :Green count: 0 :    RED
0.992301
Image: 236 :  Red count: 221 :Yellow count: 0 :Green count: 0 :    RED
0.989611
Image: 237 :  Red cou

In [ ]:
import numpy as np
import cv2

red = np.uint8([[[0, 0, 255]]])
hsvred = cv2.cvtColor(red,cv2.COLOR_BGR2HSV)
print(hsvGreen)
lowerLimit = (hsvred[0][0][0]-10,100,100)
upperLimit = (hsvred[0][0][0]+10,255,255)
print(upperLimit)
print(lowerLimit)